1. อ่าน PDF และแยกประโยคด้วย PyThaiNLP

In [ ]:
from pypdf import PdfReader
from pythainlp.tokenize import sent_tokenize

def load_pdf_as_sentences(pdf_path: str) -> list[str]:
    """
    อ่านข้อความจาก PDF → ตัดประโยคภาษาไทยด้วย CRFcut → คืนเป็นลิสต์ประโยค
    """
    reader = PdfReader(pdf_path)
    all_sentences = []

    for page in reader.pages:
        text = page.extract_text()
        if text:
            # ใช้ crfcut แทน attacut เพื่อให้ได้ประโยคที่แม่นยำ
            sents = sent_tokenize(text, engine="crfcut")
            # กรองประโยคว่าง
            sents = [s.strip() for s in sents if s.strip()]
            all_sentences.extend(sents)

    return all_sentences

if __name__ == "__main__":
    # ทดลองอ่าน PDF สองไฟล์ที่กำหนด
    sentences1 = load_pdf_as_sentences("data/537_SET ESG Ratings 2024_Announcement_final.pdf")
    sentences2 = load_pdf_as_sentences("data/736_Factsheet & FAQ SET ESG Ratings 2024.pdf")
    all_sentences = sentences1 + sentences2

    print(f"จำนวนประโยคทั้งหมด: {len(all_sentences)}")
    print("ตัวอย่างประโยค 5 ประโยคแรก:")
    for i, s in enumerate(all_sentences[:5], 1):
        print(f"{i}. {s}")


จำนวนประโยคทั้งหมด: 386
ตัวอย่างประโยค 5 ประโยคแรก:
1. การประกาศผล
SET ESG Ratings 
ประจ าปี 
2567
ฝ่ายพัฒนาการลงทุนอย่างยั่งยืน 
ตลาดหลักทรัพย์แห่งประเทศไทย
16 ธันวาคม 2567
2. ท าความรู้จัก
SET ESG Ratings
3. หุ้นของบริษัทจดทะเบียนที่ให้ความส าคัญกับสิ่งแวดล้อม สังคม และ
บรรษัทภิบาล (Environmental, Social and Governance: ESG)
ซึ่งผ่านการคัดเลือกจากตลาดหลักทรัพย์ฯ
คืออะไร
ประกาศผล
ประกาศผลปีละ 1 ครั้ง ในเดือน ธ.ค.
4. รู้จักกับ SET ESG Ratings
Copyright 2024 © The Stock Exchange of Thailand.
5. All rights reserved.


2. สร้าง Embedding ของแต่ละประโยคด้วย WangchanBERTa (Slow Tokenizer)

In [ ]:
import torch
import numpy as np
from transformers import CamembertTokenizer, AutoModel

# โหลด tokenizer (slow) + model WangchanBERTa
MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# 3.2 embedding (mean pooling)
def encode_sentences_wangchanberta(
    sentences: list[str],
    tokenizer: CamembertTokenizer,
    model: AutoModel,
    max_length: int = 128,
    batch_size: int = 16,
    device: torch.device | None = None
) -> np.ndarray:
    """
    แปลงลิสต์ประโยคเป็น embedding (mean-pooling) ด้วย WangchanBERTa
    คืน numpy array shape = (N, hidden_size) เช่น (จำนวนประโยค, 768)
    """
    all_embeddings = []
    with torch.no_grad():
        for i in range(0, len(sentences), batch_size):
            batch_sents = sentences[i : i + batch_size]
            enc = tokenizer(
                batch_sents,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden = outputs.last_hidden_state  # shape = (B, L, H)

            # สร้าง mask ให้ตำแหน่ง padding = 0
            mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()  # (B, L, H)
            masked_hidden = last_hidden * mask  # (B, L, H)

            # sum ตามแกน tokens แล้วหารด้วยจำนวน token จริง (mean pooling)
            summed = torch.sum(masked_hidden, dim=1)  # (B, H)
            counts = torch.clamp(mask.sum(dim=1), min=1e-9)[:, 0]  # (B,)
            mean_pooled = summed / counts.unsqueeze(-1)  # (B, H)

            all_embeddings.append(mean_pooled.cpu().numpy())

    all_embeddings = np.vstack(all_embeddings)  # (N, H)
    return all_embeddings.astype("float32")

# ทดลองสร้าง embedding ของประโยคตัวอย่าง
if __name__ == "__main__":
    test_sentences = [
        "บริษัทได้จัดตั้งโครงการ SET ESG Ratings ด้านการเปิดเผยข้อมูลและจัดลำดับความสำคัญของการดำเนินกิจกรรมด้านความยั่งยืน",
        "แนวคิดหลักของ ESG Ratings คือการประเมินประสิทธิภาพด้านสิ่งแวดล้อม สังคม และธรรมาภิบาล",
        "การเปิดเผยข้อมูลให้สอดคล้องกับหลักเกณฑ์ที่เป็นสากล ช่วยสร้างความเชื่อถือให้กับนักลงทุน"
    ]
    emb = encode_sentences_wangchanberta(
        sentences=test_sentences,
        tokenizer=tokenizer,
        model=model,
        max_length=128,
        batch_size=2,
        device=device
    )
    print("Shape of embeddings:", emb.shape)  # ควรเป็น (3, 768)


Shape of embeddings: (3, 768)


3. คำนวณ Cosine Similarity ระหว่างประโยคติดกัน → หา “Boundary” (จุดแบ่งหัวข้อ)

In [29]:
import numpy as np

def compute_adjacent_similarities(embeddings: np.ndarray) -> np.ndarray:
    """
    รับ embeddings shape = (N, D)
    คืน array shape = (N-1,) โดย similarities[i] = cosine(emb[i], emb[i+1])
    """
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)  # (N, 1)
    normalized = embeddings / (norms + 1e-12)
    sims = np.sum(normalized[:-1] * normalized[1:], axis=1)   # (N-1,)
    return sims

def find_boundaries(similarities: np.ndarray, threshold: float = 0.75) -> list[int]:
    """
    คืน list ของดัชนี i ที่ similarities[i] < threshold 
    (หมายความว่าประโยค i กับ i+1 เปลี่ยนหัวข้อ)
    """
    return np.where(similarities < threshold)[0].tolist()


4. สร้าง Semantic Chunks → ปรับขนาด (merge / split ตามขนาด)

In [30]:
def build_semantic_chunks(
    sentences: list[str],
    boundaries: list[int]
) -> list[list[str]]:
    """
    รับ list ประโยค และ list ดัชนี boundary
    คืนเป็น list ของ chunks (แต่ละ chunk คือ list ของประโยค)
    """
    chunks: list[list[str]] = []
    start = 0
    for b in boundaries:
        chunks.append(sentences[start : b + 1])
        start = b + 1
    if start < len(sentences):
        chunks.append(sentences[start : len(sentences)])
    return chunks

def adjust_chunks(
    raw_chunks: list[list[str]],
    min_sentences: int = 3,
    max_sentences: int = 50
) -> list[list[str]]:
    """
    - ถ้า chunk สั้นกว่า min_sentences → merge เข้ากับ chunk ก่อนหน้า (ถ้ามี)
    - ถ้า chunk ยาวกว่า max_sentences → split ออกคร่าว ๆ (แบ่งครึ่ง)
    """
    adjusted: list[list[str]] = []
    for chunk in raw_chunks:
        # merge ถ้า chunk เล็กเกินไป
        if len(chunk) < min_sentences:
            if adjusted:
                adjusted[-1].extend(chunk)
            else:
                adjusted.append(chunk)
            continue

        # split ถ้า chunk ใหญ่เกินไป
        if len(chunk) > max_sentences:
            mid = len(chunk) // 2
            adjusted.append(chunk[:mid])
            adjusted.append(chunk[mid:])
        else:
            adjusted.append(chunk)
    return adjusted


5. รวมประโยคในแต่ละ Chunk เป็นข้อความเดียว → สร้าง Embedding ของแต่ละ Chunk → เก็บลง FAISS

In [ ]:
import faiss
import pickle

if __name__ == "__main__":
    # อ่าน PDF แล้วได้ list ประโยคทั้งหมด 
    sentences1 = load_pdf_as_sentences("data/537_SET ESG Ratings 2024_Announcement_final.pdf")
    sentences2 = load_pdf_as_sentences("data/736_Factsheet & FAQ SET ESG Ratings 2024.pdf")
    all_sentences = sentences1 + sentences2

    # สร้าง embedding ของแต่ละประโยค (N × 768) 
    sentence_embeddings = encode_sentences_wangchanberta(
        sentences=all_sentences,
        tokenizer=tokenizer,
        model=model,
        max_length=128,
        batch_size=16,
        device=device
    )

    # คำนวณ similarity → หา boundary 
    sims = compute_adjacent_similarities(sentence_embeddings)
    boundaries = find_boundaries(sims, threshold=0.75)

    # สร้าง raw_chunks จากประโยค 
    raw_chunks = build_semantic_chunks(all_sentences, boundaries)

    # ปรับขนาด chunks (merge/split) เพื่อให้แต่ละก้อนเหมาะสม 
    final_chunks = adjust_chunks(raw_chunks, min_sentences=3, max_sentences=50)

    # รวมเป็นข้อความเดียว (string) ของแต่ละ chunk 
    chunk_texts = [" ".join(chunk) for chunk in final_chunks]

    print(f"จำนวนประโยคทั้งหมด = {len(all_sentences)}")
    print(f"จำนวน semantic chunks หลังปรับขนาด = {len(chunk_texts)}")

    # สร้าง embedding ของแต่ละ chunk (M × 768, เมื่อ M = จำนวน chunks) 
    chunk_embeddings = encode_sentences_wangchanberta(
        sentences=chunk_texts,
        tokenizer=tokenizer,
        model=model,
        max_length=256,    # เราเพิ่ม max_length เพราะ chunk มักยาวกว่า 1 ประโยค
        batch_size=8,
        device=device
    )
    print("Shape of chunk_embeddings:", chunk_embeddings.shape)  # (M, 768)

    # สร้าง FAISS index และเพิ่ม vectors 
    dim = chunk_embeddings.shape[1]  # 768
    faiss_index = faiss.IndexFlatL2(dim)
    faiss_index.add(chunk_embeddings)

    # บันทึก FAISS index ลงไฟล์ (.index) และ chunk_texts ลง pickle 
    faiss.write_index(faiss_index, "chunk_faiss.index")
    with open("chunk_texts.pkl", "wb") as f:
        pickle.dump(chunk_texts, f)

    print("✅ สร้าง FAISS index และบันทึก chunk_texts เรียบร้อยแล้ว")


จำนวนประโยคทั้งหมด = 386
จำนวน semantic chunks หลังปรับขนาด = 18
Shape of chunk_embeddings: (18, 768)
✅ สร้าง FAISS index และบันทึก chunk_texts เรียบร้อยแล้ว
